In [ ]:
df_elo = pd.DataFrame(columns=['Sailor', 'Teams', 'Pos', 'Elo'])

new_rows = []
for p in people.values():
    df_elo.loc[len(df_elo)] = [p.name, p.teams, p.pos, p.r.mu]
    row = {"Sailor": p.name}
    row.update(zip(p.races, [(int(c['score']), float(c['change']), float(c['regAvg']), float(c['newRating'])) for c in p.changes]))
    new_rows.append(row)

all_keys = ['Sailor'] + list(df_races['raceID'].unique())
new_df = pd.DataFrame(new_rows, columns=all_keys)

df_elo = pd.concat([df_elo.set_index('Sailor'), new_df.set_index('Sailor')], axis=1).reset_index()

# Filter sailors who have 'f24' in their seasons list
eligible_sailors = [p.name for p in people.values() if 'f24' in p.seasons]

# Filter the df_elo DataFrame to only include eligible sailors
df_elo_filtered = df_elo[df_elo['Sailor'].isin(eligible_sailors)]

# Sort the filtered df_elo by Elo and reset index
df_elo_filtered = df_elo_filtered.sort_values('Elo', ascending=False)
df_elo_filtered.reset_index(drop=True, inplace=True)

# Rank sailors within each position (Skipper/Crew) based on Elo
df_elo_filtered['Rank'] = df_elo_filtered.groupby('Pos')['Elo'].rank(method='dense', ascending=False).astype(int)

# Merge the df_elo_filtered with the sailors' data to get the rank in bulk
# df_rank = df_elo_filtered[['Sailor', 'Pos', 'Rank']]

# # Merge with people data to assign ranks in one go
# people_df = pd.DataFrame({
#     'Sailor': [p.name for p in people.values()],
#     'Pos': [p.pos for p in people.values()],
#     'rank': [None] * len(people)  # Initialize with None
# })

# Perform a merge to get the rank assigned to each sailor
# people_df = people_df.merge(df_rank, on=['Sailor', 'Pos'], how='left')

# Update the rank attribute of each sailor in the people dictionary
# for p in people.values():
    # p.rank = people_df.loc[people_df['Sailor'] == p.name, 'Rank'].values[0]


# Update rank attribute for each sailor (only for those in the filtered list)
for p in people.values():
    if p.name in eligible_sailors:
        p.rank = df_elo_filtered.loc[(df_elo_filtered['Sailor'] == p.name) & (df_elo_filtered['Pos'] == p.pos), 'Rank'].values[0]

In [ ]:
# makes bar chart
import plotly.express as px
import plotly.figure_factory as ff

# predictions = model.predict_rank([[p.r] for p in list(people.values()) if p.name in ['Carter Anderson', 'Justin Callahan', 'Blake Roberts'] and p.pos == 'Skipper'])
# print([p.name for p in list(people.values()) if p.name in ['Carter Anderson', 'Justin Callahan', 'Blake Roberts']  and p.pos == 'Skipper'])
# print(predictions)

# for raceID in list(df_races_full['raceID'].unique()):
# print(raceID)
# raceID = 'f24/open-atlantic-coast-final/1A'
raceID = "f24/mcsa-open-fall/10A"
# race = df_races_full.loc[df_races_full['raceID'] == raceID]
race = df_races_full[df_races_full['raceID'] == raceID]
df_preds3 = pd.DataFrame(columns=['pred', 'actual', 'Sailor'])
for pos in ['Skipper', 'Crew']:
    sailors = [p for p in people.values() if p.name in list(race['Sailor']) and race.loc[race['Sailor'] == p.name, 'Position'].iat[0] == pos and p.pos == pos]
    predictions = model.predict_rank([[p.r] for p in sailors])

    for pred, sailor in zip(predictions, sailors):
        score = float(race.loc[race['Sailor'] == sailor.name, 'Score'])
        # print(f"predicted: {pred[0]}, actual: {int(score)}, name: {sailor.name}")
        df_preds3.loc[len(df_preds3)] = [float(pred[0]), score, sailor.name]
    
df_preds = pd.read_csv('predsLR.csv')
df_preds2 = pd.read_csv('predsRF.csv')

df_preds3['predLR'] = df_preds['pred']
df_preds3['predRF'] = df_preds2['Predicted Score']

err = (abs(df_preds3['pred'] - df_preds3['actual'])).mean()
print(f"Mean Absolute Error for predictions in Race {raceID}: {err:.2f}")
err = (abs(df_preds3['predLR'] - df_preds3['actual'])).mean()
print(f"Mean Absolute Error for predictions in Race {raceID}: {err:.2f}")
err = (abs(df_preds3['predRF'] - df_preds3['actual'])).mean()
print(f"Mean Absolute Error for predictions in Race {raceID}: {err:.2f}")

fig = px.bar(df_preds3,x='Sailor', y=['actual', 'pred', 'predLR', 'predRF'],color_discrete_map={'actual': '#0101fd', 'pred': '#ffa500'})
fig.update_layout(width=1100, title='Model Predicted and Actual Score vs Sailor (lower is better)',
    xaxis_title='Num Teams',
    
    yaxis_title='Score',
    barmode='group')
fig.show()